In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'numpy.rec'

In [ ]:
data = pd.read_csv("../data/WEC_Perth_49.csv")

In [ ]:
coordinate_features = [f"X{i}" for i in range(1, 50)] + [f"Y{i}" for i in range(1, 50)]
power_features = [f"Power{i}" for i in range(1, 50)]
features = coordinate_features + power_features + ["qw"]
X = data[features].values
y = data["TotalPower"].values

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test).reshape(-1, 1)

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [ ]:
model = Net(X_train.shape[1])

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 100
batch_size = 32
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i : i + batch_size]
        batch_y = y_train_tensor[i : i + batch_size]

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        train_loss = criterion(model(X_train_tensor), y_train_tensor)
        val_loss = criterion(model(X_val_tensor), y_val_tensor)
        train_losses.append(train_loss.item())
        val_losses.append(val_loss.item())

    if (epoch + 1) % 10 == 0:
        print(
            f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}"
        )

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy()
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
plt.xlabel("Actual Total Power")
plt.ylabel("Predicted Total Power")
plt.title("Neural Network: Predicted vs Actual Total Power (Including XY Coordinates)")
plt.tight_layout()
plt.show()

# Plot learning curves
plt.figure(figsize=(12, 4))
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.show()

# Calculate feature importance
importance = torch.abs(model.fc1.weight).sum(dim=0).detach().numpy()
importance = importance / np.sum(importance)

# Plot feature importance
plt.figure(figsize=(20, 10))
plt.bar(features, importance)
plt.title("Neural Network Feature Importance (Including XY Coordinates)")
plt.xlabel("Features")
plt.ylabel("Importance")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Analyze spatial importance
coord_importance = importance[:98]  # First 98 features are X1-X49, Y1-Y49
x_importance = coord_importance[:49]
y_importance = coord_importance[49:]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(x_importance.reshape(7, 7), cmap="viridis")
plt.title("X Coordinate Importance")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(y_importance.reshape(7, 7), cmap="viridis")
plt.title("Y Coordinate Importance")
plt.colorbar()

plt.tight_layout()
plt.show()